1. Import functions

In [1]:
import os
import warnings
from collections import deque
from datetime import timedelta
import random
import numpy as np
import pandas as pd
from scipy.sparse import bmat, coo_matrix, csr_matrix, diags, kron, load_npz
from scipy import linalg
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.constraints import NonNeg
from tensorflow.keras.layers import Bidirectional, Dense, LSTM, TimeDistributed, Flatten, Reshape
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Model
from tabulate import tabulate
from itertools import chain
from termcolor import colored
from random import randrange
from IPython.display import clear_output
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import cupy as cp
import time
import traceback
import datetime
import math
from livelossplot import PlotLossesKeras

2. Check GPU

In [2]:
numpy_array = np.random.rand(100)
cupy_array = cp.asarray(numpy_array)
if cupy_array.data.device.id >= 0:
    print("CuPy is using the GPU.")
else:
    print("CuPy is NOT using the GPU.")

CuPy is using the GPU.


3. Necessary functions

In [3]:
#1. folding and Unfolding a 3D tensor
def unfolding_3D(Tens, unfol_dim, other_dim_seq ):
    X = Tens.transpose(unfol_dim, other_dim_seq[0], other_dim_seq[1]).reshape(Tens.shape[unfol_dim], Tens.shape[other_dim_seq[0]] * Tens.shape[other_dim_seq[1]])
    return (X)
def folding_3D(Unfold_Tens, unfol_dim , other_dim_seq, Tens_shape):
    a = [0,1,2]
    items = deque(a)
    items.rotate(unfol_dim) 
    a_up= list(items)
    X = Unfold_Tens.reshape(Tens_shape[unfol_dim], Tens_shape[other_dim_seq[0]],Tens_shape[other_dim_seq[1]]).transpose(a_up[0], a_up[1], a_up[2])    
    return (X)
#2. ADGNTD functions
def derivative_dynamic_graph_A_W(A, B, W):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    Imn = csr_matrix(np.identity(A.shape[0] * A.shape[1]))
    ones = csr_matrix(np.ones((1, A.shape[0])))
    Eta_rep = csr_matrix(linalg.khatri_rao(Imn.toarray(), (B.dot(kron(In, ones))).toarray()))
    Eta = Eta_rep.T.dot(kron(In.T, W)).dot(Eta_rep)
    derivative_A = (Eta + Eta.T).dot(vect(A))
    derivative_A_mat = vec_to_Mat(derivative_A, A.shape)
    return(csr_matrix(derivative_A_mat))
def derivative_dynamic_graph_A_D(A, B, Deg):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    Imn = csr_matrix(np.identity(A.shape[0] * A.shape[1]))
    ones = csr_matrix(np.ones((1, A.shape[0])))
    Eta_rep = csr_matrix(linalg.khatri_rao(Imn.toarray(), (B.dot(kron(In, ones))).toarray()))
    Eta = Eta_rep.T.dot(kron(In.T, Deg)).dot(Eta_rep)
    derivative_A = (Eta + Eta.T).dot(vect(A))
    derivative_A_mat = vec_to_Mat(derivative_A, A.shape)
    return(csr_matrix(derivative_A_mat))
def derivative_dynamic_graph_B_W(A, B, W):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    I_dash = diags(np.ones((W.shape[0],)))
    Xi_rep = kron(linalg.khatri_rao(In.toarray(), A.toarray()), Ip)
    Xi = Xi_rep.T.dot(kron(In.T , W)).dot(Xi_rep)
    derivative_B = (Xi + Xi.T).dot(vect(B))
    derivative_B_mat = vec_to_Mat(derivative_B, B.shape)
    return(csr_matrix(derivative_B_mat))
def derivative_dynamic_graph_B_D(A, B, Deg):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    Xi_rep = kron(linalg.khatri_rao(In.toarray(), A.toarray()), Ip)
    Xi = Xi_rep.T.dot(kron(In.T , Deg)).dot(Xi_rep)
    derivative_B = (Xi + Xi.T).dot(vect(B))
    derivative_B_mat = vec_to_Mat(derivative_B, B.shape)
    return(csr_matrix(derivative_B_mat))
def vect(Mat):
    x = Mat.shape[0]
    y = Mat.shape[1]
    vec_A = Mat.T.reshape(x*y,1)
    return(vec_A)
def vec_to_Mat(vec, shape):
    x = shape[0]
    y = shape[1]
    Mat = vec.reshape(y,x).T
    return (Mat)
def rmse(a, b):
    rms = np.sqrt(np.power(a-b, 2).sum()/(a.shape[0]* a.shape[1]))
    return (round(rms,2))
    
def calculate_smoothed_weights(mat):
    identity_matrix = np.identity(mat.shape[0])
    W_smooth = -pd.DataFrame(identity_matrix).rolling(2, axis=1).sum().replace(np.nan, 0)
    W_smooth[0] = -pd.DataFrame(identity_matrix)[0]
    W_smooth = np.array(W_smooth + 2 * pd.DataFrame(identity_matrix))
    W_smooth_pos = (np.abs(W_smooth) + W_smooth) / 2
    W_smooth_neg = (np.abs(W_smooth) - W_smooth) / 2
    neum_term_B = csr_matrix(W_smooth_neg.T.dot(W_smooth_neg))
    denom_term_B = csr_matrix(W_smooth_pos.T.dot(W_smooth_pos))
    return  (neum_term_B, denom_term_B)

def calculate_smoothed_weights_cp(mat):
    identity_matrix = np.identity(mat.shape[0])
    W_smooth = -pd.DataFrame(identity_matrix).rolling(2, axis=1).sum().replace(np.nan, 0)
    W_smooth[0] = -pd.DataFrame(identity_matrix)[0]
    W_smooth = np.array(W_smooth + 2 * pd.DataFrame(identity_matrix))
    W_smooth_pos = (np.abs(W_smooth) + W_smooth) / 2
    W_smooth_neg = (np.abs(W_smooth) - W_smooth) / 2
    neum_term_B = cp.asarray(W_smooth_neg.T.dot(W_smooth_neg))
    denom_term_B = cp.asarray(W_smooth_pos.T.dot(W_smooth_pos))
    return  (neum_term_B, denom_term_B)

In [4]:
def ADGNTD(Tens1, W_prior, r = 10, n_iter=100, hy1=1, hy2=1, mu=0.1, factorization1=1000,
                      prin=50, A = None, B = None, C = None, W = None, delta = None ):
    T3 = csr_matrix(unfolding_3D(Tens1, unfol_dim = 2, other_dim_seq = [0,1] ))
    plot = []
    try:
        if (A == None) | (B == None)|  (C == None):
            A = csr_matrix(np.random.rand(Tens1.shape[0],r))
            B = csr_matrix(np.random.rand(Tens1.shape[1],r))
            C = csr_matrix(np.random.rand(Tens1.shape[2],r))
        if (W == None):
            W = W_prior.copy()
            delta = W*.5
        if (delta == None):
            delta = W*.5
        A_list = list(range(A.shape[0]))
        B_list = list(range(B.shape[0]))
        C_list = list(range(C.shape[0]))
        neum_term_B, denom_term_B = calculate_smoothed_weights(B)
        for indomie in range(n_iter):
            A_Batch = Tens1.shape[0] // 1
            B_Batch = Tens1.shape[1] // 1
            C_Batch = Tens1.shape[2] // 1
            A_int = random.randrange(A.shape[0])
            B_int = random.randrange(B.shape[0])
            C_int = random.randrange(C.shape[0])
            A_Sampling = random.sample(A_list, A_Batch)
            B_Sampling = random.sample(B_list, B_Batch)
            C_Sampling = random.sample(C_list, C_Batch)
            A_Sampled = csr_matrix(A[A_Sampling, :])
            B_Sampled = csr_matrix(B[B_Sampling, :])
            C_Sampled = csr_matrix(C[C_Sampling, :])
            Tens_sample = Tens1[A_Sampling, :, :][:, B_Sampling, :][:, :, C_Sampling]
            T1_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=0, other_dim_seq=[1, 2]))
            T2_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=1, other_dim_seq=[2, 0]))
            T3_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=2, other_dim_seq=[0, 1]))
            # Loop through B_Sampling indices and create w_list_B
            for index, j in enumerate(B_Sampling):
                if index == 0:
                    w_list_B = list(range(j * A.shape[0], (j + 1) * A.shape[0]))
                if index > 0:
                    w_list_B += list(range(j * A.shape[0], (j + 1) * A.shape[0]))
            
            # Loop through B indices and create w_list_A
            for i in range(0, B.shape[0]):
                if i == 0:
                    w_list_A = list(np.array(A_Sampling) + i * A.shape[0])
                if i > 0:
                    w_list_A += list(np.array(A_Sampling) + i * A.shape[0])
            
            # Find common indices between w_list_A and w_list_B
            w_list = list(set(w_list_A) & set(w_list_B))
            # Sample submatrix W_sampled from W using w_list indices
            W_sampled = W[w_list, :][:, w_list]
            # Calculate degree matrix for sampled nodes
            Deg_sampled = sparse.diags(np.squeeze(np.asarray(W_sampled.sum(axis=0)))).tocsr()
            # Sample submatrix W_prior_sampled from W_prior using w_list indices
            W_prior_sampled = W_prior[w_list, :][:, w_list]
            # Sample submatrix delta_sampled from delta using w_list indices
            delta_sampled = delta[w_list, :][:, w_list]
            # Create matrix V using Khatri-Rao product of B_Sampled and C_Sampled
            V = csr_matrix(linalg.khatri_rao(B_Sampled.toarray(), C_Sampled.toarray()))
            # Update A_Sampled using optimization step
            A_Sampled = csr_matrix(A_Sampled.multiply(T1_sampled.dot(V) 
                                                      + hy1 * derivative_dynamic_graph_A_W(A_Sampled, B_Sampled, W_sampled)) / 
                                   (A_Sampled.dot(V.T).dot(V) + hy2 * A_Sampled + 
                                    hy1 * derivative_dynamic_graph_A_D(A_Sampled, B_Sampled, Deg_sampled)))
            # Update rows of A using A_Sampling indices
            A[A_Sampling, :] = A_Sampled
            # Recalculate matrix V using Khatri-Rao product of C_Sampled and A_Sampled
            V = csr_matrix(linalg.khatri_rao(C_Sampled.toarray(), A_Sampled.toarray()))
            # Update B_Sampled using optimization step
            B_num = B_Sampled.multiply(csr_matrix(T2_sampled.dot(V)) + 5 * (neum_term_B.dot(B)) 
                                       + hy1 * derivative_dynamic_graph_B_W(A_Sampled, B_Sampled, W_sampled))
            B_denom = (B_Sampled.dot(V.T).dot(V) + hy2 * B_Sampled + 5 * (denom_term_B.dot(B)) 
                       + hy1 * derivative_dynamic_graph_B_D(A_Sampled, B_Sampled, Deg_sampled))
            B_Sampled = csr_matrix(B_num / B_denom)
            # Update rows of B using B_Sampling indices
            B[B_Sampling, :] = B_Sampled
            # Recalculate matrix V using Khatri-Rao product of A_Sampled and B_Sampled
            V = csr_matrix(linalg.khatri_rao(A_Sampled.toarray(), B_Sampled.toarray()))
            # Update C_Sampled using optimization step
            C_Sampled = csr_matrix(C_Sampled.multiply(T3_sampled.dot(V)) / (C_Sampled.dot(V.T).dot(V) + hy2 * C_Sampled))
            # Update rows of C using C_Sampling indices
            C[C_Sampling, :] = C_Sampled
            # Estimate matrix EstimatedT4 using calculated matrices
            EstimatedT4 = C.todense().dot(linalg.khatri_rao(A.todense(), B.todense()).T)
            # Update delta_sampled with mu term
            delta_sampled = delta_sampled + mu * (delta_sampled.T - delta_sampled)
            # Update submatrix delta using delta_sampled and w_list indices
            delta[w_list, :][:, w_list] = delta_sampled
            if indomie%prin == 0:
                try:
                    factorization1 =  min(factorization, factorization1)
                except:
                    pass
                factorization = rmse(T3, EstimatedT4).round(2)
                plot.append([indomie, factorization])
                abcd = pd.DataFrame(plot, columns = ['iteration', 'RMSE'])
            if indomie%prin == 0:
                clear_output(wait=True)
                print(".............................................")
                print('iteration:                       ',indomie)
                print('rmse factorization :             ', factorization )
                try:
                    print('rmse factorization min:          ', min(factorization, factorization1))
                except:
                    pass
                print('.............................................')
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        return(A, B, C, W, delta)
    return(A, B, C, W, delta)

def ADGNTD_stochastic(Tens1, W_prior, r = 10, iter_steps = [1000, 2000, 3000],
                      batchsize = [[20, 100, 20], [40,100,40], [60,100,60]], 
                      hy1=1, hy2=1, mu=0.1, factorization1=1000, max_iter = 6000,
                      prin=50, A = None, B = None, C = None, W = None, delta = None ):
    Tens1 = cp.asarray(Tens1)
    W_prior = cp.asarray(W_prior)
    def unfolding_3D(Tens, unfol_dim, other_dim_seq):
        # Convert the input tensor to CuPy
        
        # Transpose the tensor using CuPy's transpose
        X = cp.transpose(Tens_cp, (unfol_dim, other_dim_seq[0], other_dim_seq[1]))
        # Reshape the tensor using CuPy's reshape
        X = cp.reshape(X, (X.shape[unfol_dim], X.shape[other_dim_seq[0]] * X.shape[other_dim_seq[1]]))
        return X

    T3 = csr_matrix(unfolding_3D(Tens1, unfol_dim = 2, other_dim_seq = [0,1] ))
    plot = []
    try:
        if (A == None) | (B == None)|  (C == None):
            A = csr_matrix(np.random.rand(Tens1.shape[0],r))
            B = csr_matrix(np.random.rand(Tens1.shape[1],r))
            C = csr_matrix(np.random.rand(Tens1.shape[2],r))
        if (W == None):
            W = W_prior.copy()
            delta = W*.5
        if (delta == None):
            delta = W*.5
        A_list = list(range(A.shape[0]))
        B_list = list(range(B.shape[0]))
        C_list = list(range(C.shape[0]))
        neum_term_B, denom_term_B = calculate_smoothed_weights(B)
        start = 0
        for indomie in range(max_iter):
            try:
                change_iter = iter_steps[start]
            except:
                change_iter = 100000000
            if indomie >= change_iter:
                start = start+1
                #print('yes')
            if indomie <= change_iter:
                try:
                    batchsize_selected = batchsize [start]
                except:
                    pass
            #print(batchsize_selected)
            A_Batch = int(batchsize_selected[0] / 100 * Tens1.shape[0] )
            B_Batch = Tens1.shape[1]
            C_Batch = int(batchsize_selected[2] / 100 * Tens1.shape[2] )
            A_int = random.randrange(A.shape[0])
            B_int = random.randrange(B.shape[0])
            C_int = random.randrange(C.shape[0])
            A_Sampling = random.sample(A_list, A_Batch)
            B_Sampling = random.sample(B_list, B_Batch)
            C_Sampling = random.sample(C_list, C_Batch)
            A_Sampled = csr_matrix(A[A_Sampling, :])
            B_Sampled = csr_matrix(B[B_Sampling, :])
            C_Sampled = csr_matrix(C[C_Sampling, :])
            Tens_sample = Tens1[A_Sampling, :, :][:, B_Sampling, :][:, :, C_Sampling]
            T1_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=0, other_dim_seq=[1, 2]))
            T2_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=1, other_dim_seq=[2, 0]))
            T3_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=2, other_dim_seq=[0, 1]))
            # Loop through B_Sampling indices and create w_list_B
            for index, j in enumerate(B_Sampling):
                if index == 0:
                    w_list_B = list(range(j * A.shape[0], (j + 1) * A.shape[0]))
                if index > 0:
                    w_list_B += list(range(j * A.shape[0], (j + 1) * A.shape[0]))
            
            # Loop through B indices and create w_list_A
            for i in range(0, B.shape[0]):
                if i == 0:
                    w_list_A = list(np.array(A_Sampling) + i * A.shape[0])
                if i > 0:
                    w_list_A += list(np.array(A_Sampling) + i * A.shape[0])
            
            # Find common indices between w_list_A and w_list_B
            w_list = list(set(w_list_A) & set(w_list_B))
            # Sample submatrix W_sampled from W using w_list indices
            W_sampled = W[w_list, :][:, w_list]
            # Calculate degree matrix for sampled nodes
            Deg_sampled = sparse.diags(np.squeeze(np.asarray(W_sampled.sum(axis=0)))).tocsr()
            # Sample submatrix W_prior_sampled from W_prior using w_list indices
            W_prior_sampled = W_prior[w_list, :][:, w_list]
            # Sample submatrix delta_sampled from delta using w_list indices
            delta_sampled = delta[w_list, :][:, w_list]
            # Create matrix V using Khatri-Rao product of B_Sampled and C_Sampled
            V = csr_matrix(linalg.khatri_rao(B_Sampled.toarray(), C_Sampled.toarray()))
            # Update A_Sampled using optimization step
            A_Sampled = csr_matrix(A_Sampled.multiply(T1_sampled.dot(V) 
                                                      + hy1 * derivative_dynamic_graph_A_W(A_Sampled, B_Sampled, W_sampled)) / 
                                   (A_Sampled.dot(V.T).dot(V) + hy2 * A_Sampled + 
                                    hy1 * derivative_dynamic_graph_A_D(A_Sampled, B_Sampled, Deg_sampled)))
            # Update rows of A using A_Sampling indices
            A[A_Sampling, :] = A_Sampled
            # Recalculate matrix V using Khatri-Rao product of C_Sampled and A_Sampled
            V = csr_matrix(linalg.khatri_rao(C_Sampled.toarray(), A_Sampled.toarray()))
            # Update B_Sampled using optimization step
            B_num = B_Sampled.multiply(csr_matrix(T2_sampled.dot(V)) + 5 * (neum_term_B.dot(B)) 
                                       + hy1 * derivative_dynamic_graph_B_W(A_Sampled, B_Sampled, W_sampled))
            B_denom = (B_Sampled.dot(V.T).dot(V) + hy2 * B_Sampled + 5 * (denom_term_B.dot(B)) 
                       + hy1 * derivative_dynamic_graph_B_D(A_Sampled, B_Sampled, Deg_sampled))
            B_Sampled = csr_matrix(B_num / B_denom)
            # Update rows of B using B_Sampling indices
            B[B_Sampling, :] = B_Sampled
            # Recalculate matrix V using Khatri-Rao product of A_Sampled and B_Sampled
            V = csr_matrix(linalg.khatri_rao(A_Sampled.toarray(), B_Sampled.toarray()))
            # Update C_Sampled using optimization step
            C_Sampled = csr_matrix(C_Sampled.multiply(T3_sampled.dot(V)) / (C_Sampled.dot(V.T).dot(V) + hy2 * C_Sampled))
            # Update rows of C using C_Sampling indices
            C[C_Sampling, :] = C_Sampled
            # Estimate matrix EstimatedT4 using calculated matrices
            EstimatedT4 = C.todense().dot(linalg.khatri_rao(A.todense(), B.todense()).T)
            # Update delta_sampled with mu term
            
            feature_Matrix = csr_matrix(linalg.khatri_rao(A_Sampled.toarray(), B_Sampled.toarray()))
            #print((Lambda_priorW * W_prior).shape)
            #print((Lambda_adaptive * .5* feature_Matrix.dot(feature_Matrix.T)).shape) 
            #print(Lambda_priorW * W_prior + Lambda_adaptive * .5* feature_Matrix.dot(feature_Matrix.T)) 
            Deg1, W1 = adaptive_Adjacency_aug(W_sampled, W_prior_sampled, feature_Matrix, mu, delta_sampled, Lambda_adaptive, Lambda_priorW , epsilon )
            #print(C.shape, A.shape, B.shape)
            W[w_list, :][:, w_list] = W1
            Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis = 0)))).tocsr()
            if indomie%prin == 0:
                try:
                    factorization1 =  min(factorization, factorization1)
                except:
                    pass
                factorization = rmse(T3, EstimatedT4).round(2)
                plot.append([indomie, factorization])
                abcd = pd.DataFrame(plot, columns = ['iteration', 'RMSE'])
            if indomie%prin == 0:
                clear_output(wait=True)
                print(".............................................")
                print('iteration:                       ',indomie)
                print('rmse factorization :             ', factorization )
                try:
                    print('rmse factorization min:          ', min(factorization, factorization1))
                    print('batchsize_selected:          ', batchsize_selected)
                    #batchsize_selected
                except:
                    pass
                print('.............................................')
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        return(A, B, C, W, delta)
        
    return(A, B, C, W, delta)
def adaptive_Adjacency_aug(W, W_prior, feature_Matrix, mu, delta, Lambda_adaptive, Lambda_priorW , epsilon ):
    neum = csr_matrix(Lambda_priorW * W_prior + Lambda_adaptive * .5* feature_Matrix.dot(feature_Matrix.T)) #OK 
    One_n = csr_matrix(np.ones(W.shape[0]).reshape(W.shape[0], 1))
    Identity_r = csr_matrix(np.identity(feature_Matrix.shape[1]))
    Identity_n = csr_matrix(np.identity(W.shape[0]))
    Third_term = csr_matrix((feature_Matrix.dot(Identity_r).dot(feature_Matrix.T).multiply(Identity_n) ).dot(One_n).dot(One_n.T)) 
    denom =  Lambda_adaptive * .5* Third_term + Lambda_priorW * W #+ .5* np.identity(W.shape[0]) 
    temp = mu * (W-W.T) + delta - delta.T
    temp_pos = (abs(temp) + temp)/2
    temp_neg = (abs(temp) - temp)/2
    W = W.multiply(csr_matrix((neum  + temp_neg).todense() + epsilon)/(csr_matrix((denom + temp_pos).todense() + epsilon)))
    W = ((W + W.T)/2)
    #W = W.toarray()
    Deg1 = sparse.diags(np.squeeze(np.asarray(W.sum(axis = 0)))).tocsr()
    #W = csr_matrix(W)
    #Deg1 = csr_matrix(Deg1)
    return (Deg1, W)

In [5]:
import cupy as cp
from cupy.sparse import csr_matrix, kron
from cupy.sparse import identity
import scipy.sparse as sp
def ADGNTD_stochastic_cp(Tens1, W_prior, r = 10, iter_steps = [1000, 2000, 3000],
                      batchsize = [[20, 100, 20], [40,100,40], [60,100,60]], 
                      hy1=1, hy2=1, mu=0.1, factorization1=1000, max_iter = 6000,
                      prin=50, A = None, B = None, C = None, W = None, delta = None ):
    Tens1 = cp.asarray(Tens1)
    W_prior = csr_matrix(W_prior)
    def Khatri_Rao_Product(A1, B1):
        result_list = [i * A1.shape[1] + i for i in range(B1.shape[1])]
        temp = csr_matrix(kron(A1, B1))
        temp = temp[:,result_list ]
        return(temp)

    def unfolding_3D(Tens, unfol_dim, other_dim_seq ):
        X = Tens.transpose(unfol_dim, other_dim_seq[0], other_dim_seq[1]).reshape(Tens.shape[unfol_dim], Tens.shape[other_dim_seq[0]] * Tens.shape[other_dim_seq[1]])
        return (X)


    def calculate_smoothed_weights(mat):
        identity_matrix = np.identity(mat.shape[0])
        W_smooth = -pd.DataFrame(identity_matrix).rolling(2, axis=1).sum().replace(np.nan, 0)
        W_smooth[0] = -pd.DataFrame(identity_matrix)[0]
        W_smooth = np.array(W_smooth + 2 * pd.DataFrame(identity_matrix))
        W_smooth_pos = (np.abs(W_smooth) + W_smooth) / 2
        W_smooth_neg = (np.abs(W_smooth) - W_smooth) / 2
        neum_term_B = cp.asarray(W_smooth_neg.T.dot(W_smooth_neg))
        denom_term_B = cp.asarray(W_smooth_pos.T.dot(W_smooth_pos))
        return  (csr_matrix(neum_term_B), csr_matrix(denom_term_B))

    def vec_to_Mat(vec, shape):
        x = shape[0]
        y = shape[1]
        Mat = vec.reshape(y,x).T
        return (Mat)
    
    def derivative_dynamic_graph_B_W(A, B, W):
        scipy_csr_identity = identity(W.shape[0], format='csr')
        I_dash = cp.sparse.csr_matrix(scipy_csr_identity)
        In = csr_matrix(cp.identity(A.shape[1]))
        Ip = csr_matrix(cp.identity(B.shape[0]))
        Xi_rep = kron(Khatri_Rao_Product(In, A), Ip)
        kron_product = kron(In.T , W)
        Xi = Xi_rep.T.dot(kron_product).dot(Xi_rep)
        derivative_B = (Xi + Xi.T).dot(vect(B))
        derivative_B_mat = vec_to_Mat(derivative_B, B.shape)
        return(csr_matrix(derivative_B_mat))
    
    def derivative_dynamic_graph_B_D(A, B, Deg):
        In = csr_matrix(cp.identity(A.shape[1]))
        Ip = csr_matrix(cp.identity(B.shape[0]))
        Xi_rep = kron(Khatri_Rao_Product(In, A), Ip)
        Xi = Xi_rep.T.dot(kron(In.T , Deg)).dot(Xi_rep)
        derivative_B = (Xi + Xi.T).dot(vect(B))
        derivative_B_mat = vec_to_Mat(derivative_B, B.shape)
        return(csr_matrix(derivative_B_mat))
    
    def derivative_dynamic_graph_A_W(A, B, W):
        In = csr_matrix(cp.identity(A.shape[1]))
        Ip = csr_matrix(cp.identity(B.shape[0]))
        Imn = identity(A.shape[0] * A.shape[1], format='csr')
        
        ones = csr_matrix(cp.ones((1, A.shape[0])))
        #print(type(B), type(Imn), type(In), type(ones))
        #tensor_product(Imn, (B.dot(kron(In, ones))))
        Eta_rep = csr_matrix(Khatri_Rao_Product(Imn, (B.dot(kron(In, ones)))))
        Eta = Eta_rep.T.dot(kron(In.T, W)).dot(Eta_rep)
        derivative_A = (Eta + Eta.T).dot(vect(A))
        derivative_A_mat = vec_to_Mat(derivative_A, A.shape)
        return(csr_matrix(derivative_A_mat))
    
    def derivative_dynamic_graph_A_D(A, B, Deg):
        In = csr_matrix(cp.identity(A.shape[1]))
        Ip = csr_matrix(cp.identity(B.shape[0]))
        Imn = csr_matrix(cp.identity(A.shape[0] * A.shape[1]))
        ones = csr_matrix(cp.ones((1, A.shape[0])))
        Eta_rep = csr_matrix(Khatri_Rao_Product(Imn, (B.dot(kron(In, ones)))))
        Eta = Eta_rep.T.dot(kron(In.T, Deg)).dot(Eta_rep)
        derivative_A = (Eta + Eta.T).dot(vect(A))
        derivative_A_mat = vec_to_Mat(derivative_A, A.shape)
        return(csr_matrix(derivative_A_mat))
    def calculate_rmse(y_true, y_pred):
        squared_diff = (y_true - y_pred) ** 2
        mean_squared_diff = cp.mean(squared_diff)
        rmse = cp.sqrt(mean_squared_diff)
        return rmse.item()
    T3 = csr_matrix(unfolding_3D(Tens1, unfol_dim = 2, other_dim_seq = [0,1] ))
    plot = []
    try:
        if (A is None) | (B is None)|  (C is None):
            A = csr_matrix(cp.random.rand(Tens1.shape[0],r))
            B = csr_matrix(cp.random.rand(Tens1.shape[1],r))
            C = csr_matrix(cp.random.rand(Tens1.shape[2],r))
        if (W is None):
            W = W_prior.copy()
            delta = W*.5
        if delta is None:
            delta = W*.5
        A_list = list(range(A.shape[0]))
        B_list = list(range(B.shape[0]))
        C_list = list(range(C.shape[0]))
        neum_term_B, denom_term_B = calculate_smoothed_weights(B)
        start = 0
        for indomie in range(max_iter):
            try:
                change_iter = iter_steps[start]
            except:
                change_iter = 100000000
            if indomie >= change_iter:
                start = start+1
                #print('yes')
            if indomie <= change_iter:
                try:
                    batchsize_selected = batchsize [start]
                except:
                    pass
            #print(batchsize_selected)
            A_Batch = int(batchsize_selected[0] / 100 * Tens1.shape[0] )
            B_Batch = int(batchsize_selected[0] / 100 * Tens1.shape[1] )
            #print(B_Batch)
            C_Batch = int(batchsize_selected[2] / 100 * Tens1.shape[2] )
            A_int = random.randrange(A.shape[0])
            B_int = random.randrange(B.shape[0])
            C_int = random.randrange(C.shape[0])
            A_Sampling = random.sample(A_list, A_Batch)
            B_Sampling = random.sample(B_list, B_Batch)
            C_Sampling = random.sample(C_list, C_Batch)
            A_Sampled = csr_matrix(A[A_Sampling, :])
            B_Sampled = csr_matrix(B[B_Sampling, :])
            C_Sampled = csr_matrix(C[C_Sampling, :])
            Tens_sample = Tens1[A_Sampling, :, :][:, B_Sampling, :][:, :, C_Sampling]
            #print(Tens_sample.shape)
            T1_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=0, other_dim_seq=[1, 2]))
            T2_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=1, other_dim_seq=[2, 0]))
            T3_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=2, other_dim_seq=[0, 1]))
            # Loop through B_Sampling indices and create w_list_B
            for index, j in enumerate(B_Sampling):
                if index == 0:
                    w_list_B = list(range(j * A.shape[0], (j + 1) * A.shape[0]))
                if index > 0:
                    w_list_B += list(range(j * A.shape[0], (j + 1) * A.shape[0]))
            
            # Loop through B indices and create w_list_A
            for i in range(0, B.shape[0]):
                if i == 0:
                    w_list_A = list(np.array(A_Sampling) + i * A.shape[0])
                if i > 0:
                    w_list_A += list(np.array(A_Sampling) + i * A.shape[0])
            #print(w_list_A, type(w_list_A), w_list_B, type(w_list_B))
            # Find common indices between w_list_A and w_list_B
            w_list = list(set(w_list_A) & set(w_list_B))
            # Sample submatrix W_sampled from W using w_list indices
            W_sampled = W[w_list, :][:, w_list]
            #print(W_sampled.shape)
            # Calculate degree matrix for sampled nodes
            Deg_sampled = csr_matrix(cp.diag(cp.sum(W_sampled.toarray(), axis=1).ravel()))
            # Sample submatrix W_prior_sampled from W_prior using w_list indices
            W_prior_sampled = W_prior[w_list, :][:, w_list]
            # Sample submatrix delta_sampled from delta using w_list indices
            delta_sampled = delta[w_list, :][:, w_list]
            # Create matrix V using Khatri-Rao product of B_Sampled and C_Sampled
            V = csr_matrix(Khatri_Rao_Product(B_Sampled, C_Sampled))
            # Update A_Sampled using optimization step
            #print(type(V), type(T1_sampled))
            A_num = A_Sampled.multiply(T1_sampled.dot(V) + hy1 * derivative_dynamic_graph_A_W(A_Sampled, B_Sampled, W_sampled))
            A_denom = A_Sampled.dot(V.T).dot(V) + hy2 * A_Sampled + hy1 * derivative_dynamic_graph_A_D(A_Sampled, B_Sampled, Deg_sampled)
            A_num.data += 0.000000001
            A_denom.data += 0.000000001
            A_Sampled = csr_matrix(A_num / A_denom)
            # Update rows of A using A_Sampling indices
            A[A_Sampling, :] = A_Sampled
            # Recalculate matrix V using Khatri-Rao product of C_Sampled and A_Sampled
            V = csr_matrix(Khatri_Rao_Product(C_Sampled, A_Sampled))
            # Update B_Sampled using optimization step
            #print(T2_sampled.shape, V.shape, type(T2_sampled), type(V), neum_term_B.shape, type(neum_term_B))
            neum_term_B, denom_term_B = calculate_smoothed_weights(B)
            neum_term_B = neum_term_B[B_Sampling, :][:, B_Sampling]
            denom_term_B = denom_term_B[B_Sampling, :][:, B_Sampling]
            B_num = B_Sampled.multiply(csr_matrix(T2_sampled.dot(V)) + 5 * (neum_term_B.dot(B_Sampled)) + hy1 * derivative_dynamic_graph_B_W(A_Sampled, B_Sampled, W_sampled))
            B_num.data += 0.000000001
            B_denom = (B_Sampled.dot(V.T).dot(V) + hy2 * B_Sampled + 5 * (denom_term_B.dot(B_Sampled)) + hy1 * derivative_dynamic_graph_B_D(A_Sampled, B_Sampled, Deg_sampled))
            B_denom.data += 0.000000001
            B_Sampled = csr_matrix(B_num / B_denom)
            # Update rows of B using B_Sampling indices
            B[B_Sampling, :] = B_Sampled
            # Recalculate matrix V using Khatri-Rao product of A_Sampled and B_Sampled
            V = csr_matrix(Khatri_Rao_Product(A_Sampled, B_Sampled))
            # Update C_Sampled using optimization step
            C_num = C_Sampled.multiply(T3_sampled.dot(V))
            C_denom = (C_Sampled.dot(V.T).dot(V) + hy2 * C_Sampled)
            C_num.data += 0.000000001
            C_denom.data += 0.000000001
            C_Sampled = C_num/ C_denom
            # Update rows of C using C_Sampling indices
            C[C_Sampling, :] = C_Sampled
            # Estimate matrix EstimatedT4 using calculated matrices
            EstimatedT4 = C.dot(Khatri_Rao_Product(A, B).T)
            # Update delta_sampled with mu term
            
            feature_Matrix = csr_matrix(Khatri_Rao_Product(A_Sampled, B_Sampled))
            #print((Lambda_priorW * W_prior).shape)
            #print((Lambda_adaptive * .5* feature_Matrix.dot(feature_Matrix.T)).shape) 
            #print(Lambda_priorW * W_prior + Lambda_adaptive * .5* feature_Matrix.dot(feature_Matrix.T)) 
            Deg1, W1 = adaptive_Adjacency_aug_cp(W_sampled, W_prior_sampled, feature_Matrix, mu, delta_sampled, Lambda_adaptive, Lambda_priorW , epsilon )
            #print(C.shape, A.shape, B.shape)
            W[w_list, :][:, w_list] = W1
            Deg = cp.sparse.diags(cp.squeeze(cp.asarray(W.sum(axis = 0)))).tocsr()
            if indomie%prin == 0:
                try:
                    factorization1 =  min(factorization, factorization1)
                except:
                    pass
                factorization = calculate_rmse(T3.toarray(), EstimatedT4.toarray())
                plot.append([indomie, factorization])
                abcd = pd.DataFrame(plot, columns = ['iteration', 'RMSE'])
            if indomie%prin == 0:
                clear_output(wait=True)
                print(".............................................")
                print('iteration:                       ',indomie)
                print('rmse factorization :             ', factorization )
                try:
                    print('rmse factorization min:          ', min(factorization, factorization1))
                    print('batchsize_selected:          ', batchsize_selected)
                    #batchsize_selected
                except:
                    pass
                print('.............................................')
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        return(A, B, C, W, delta)
        
    return(A, B, C, W, delta)
def adaptive_Adjacency_aug_cp(W, W_prior, feature_Matrix, mu, delta, Lambda_adaptive, Lambda_priorW , epsilon ):
    neum = csr_matrix(Lambda_priorW * W_prior + Lambda_adaptive * .5* feature_Matrix.dot(feature_Matrix.T)) #OK 
    One_n = csr_matrix(cp.ones(W.shape[0]).reshape(W.shape[0], 1))
    Identity_r = csr_matrix(cp.identity(feature_Matrix.shape[1]))
    Identity_n = csr_matrix(cp.identity(W.shape[0]))
    Third_term = csr_matrix((feature_Matrix.dot(Identity_r).dot(feature_Matrix.T).multiply(Identity_n) ).dot(One_n).dot(One_n.T)) 
    denom =  Lambda_adaptive * .5* Third_term + Lambda_priorW * W #+ .5* cp.identity(W.shape[0]) 
    temp = mu * (W-W.T) + delta - delta.T
    temp_pos = (abs(temp) + temp)/2
    temp_neg = (abs(temp) - temp)/2
    W = W.multiply(csr_matrix((neum  + temp_neg).todense() + epsilon)/(csr_matrix((denom + temp_pos).todense() + epsilon)))
    W = ((W + W.T)/2)
    #W = W.toarray()
    Deg1 = csr_matrix(cp.diag(cp.sum(W.toarray(), axis=1).ravel()))
    #Deg1 = sparse.diags(cp.squeeze(cp.asarray(W.sum(axis = 0)))).tocsr()
    #W = csr_matrix(W)
    #Deg1 = csr_matrix(Deg1)
    return (Deg1, W)

In [6]:
import os
from scipy import sparse
# Set up data directory
name = 'Logan'
data_directory = r'\\hpc-fs.qut.edu.au\n10680535\Paper_5\Datasets'

# Load data
Tens = np.load(os.path.join(data_directory, f'Train_{name}_3D_data.npy'))
Test_Tens = np.load(os.path.join(data_directory, f'Test_{name}_3D_data.npy'))
test_dataset = pd.read_csv(os.path.join(data_directory, f'test_{name}_data.csv'), index_col=[0, 1], header=[0])
Val_Tens = np.load(os.path.join(data_directory, f'Val_{name}_3D_data.npy'))
val_dataset = pd.read_csv(os.path.join(data_directory, f'Val_{name}_data.csv'), index_col=[0, 1], header=[0])
ind = pd.read_csv(os.path.join(data_directory, 'date_time_train.csv'), index_col=[0, 1])
stl_clm = pd.read_csv(os.path.join(data_directory, 'section_names.csv'), index_col=0)
train_data_shubham = pd.read_csv(os.path.join(data_directory, f'Train_{name}_data.csv'), index_col=[0, 1], header=[0])
W = sparse.load_npz(os.path.join(data_directory, f'Dynamic_Adjacency_{name}.npz'))
MWY_Sections = pd.read_csv(os.path.join(data_directory, 'MWY_Sections.csv'))

# Extract section IDs
section_ids = train_data_shubham.columns.to_list()
serial_numbers = [test_dataset.columns.get_loc(section_id) for section_id in section_ids]

# Manipulate data based on section IDs
Tens = Tens[serial_numbers, :, :]
Test_Tens = Test_Tens[serial_numbers, :, :]
Val_Tens = Val_Tens[serial_numbers, :, :]
stl_clm = stl_clm.iloc[serial_numbers]
train_data_shubham = train_data_shubham.T.iloc[serial_numbers].T
test_dataset = test_dataset.T.iloc[serial_numbers].T
val_dataset = val_dataset.T.iloc[serial_numbers].T

# Additional data manipulations
stl_clm.index = stl_clm.index.astype('str')
test_dataset = test_dataset[stl_clm.index].T
test_data_prior = test_dataset.copy(deep=True)
val_dataset = val_dataset[stl_clm.index].T
val_data_prior = val_dataset.copy(deep=True)
test_dataset_col = test_dataset.columns
val_dataset_col = val_dataset.columns
train_data_shubham = train_data_shubham[test_data_prior.index.astype(str)]
train_data_shubham_prior = train_data_shubham.T

# Update serial numbers for W
serial_numbers_W = serial_numbers.copy()
for i in range(1, 96):
    for j in serial_numbers:
        serial_numbers_W.append(i * 476 + j)

# Update adjacency matrix W
W = coo_matrix(W.toarray()[serial_numbers_W].T[serial_numbers_W].T)
W = coo_matrix(W)
W_dense = W.toarray()
W = (W + W.T) / 2
W.setdiag(0, k=0)
Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis=0)))).tocsr()

# File path for complete data
file_path = os.path.join(data_directory, 'Complete_Logan_3D_data.npy')
Complete_Tens = np.load(file_path)
Complete_Tens = Complete_Tens[serial_numbers, :, :]

# Additional data transformation
Tens1 = Tens.transpose(0, 2, 1)

In [ ]:
r = rank = 20
Lambda_adaptive = 0.0001
Lambda_priorW = 0.01
epsilon   = 0.0000001
hy1=.001
A_L, B_L, C_L, W, delta = ADGNTD_stochastic_cp(Tens1, W, r = r, iter_steps = [6000],
                      batchsize = [[20, 100, 20]], 
                      hy1= hy1, hy2=.0001, mu=0.1, factorization1=1000,
                      prin=5, A = None, B = None, C = None, W = None, delta = None )

11. Test code

ADGNTD Entire

In [7]:
A =  A_L.copy().toarray()
B =  C_L.copy().toarray()
C =  B_L.copy().toarray()

NameError: name 'A_L' is not defined

In [ ]:
import scipy
np.save('A20.npy', A)
np.save('B20.npy', B)
np.save('C20.npy', C)
scipy.sparse.save_npz('W_s.npz', W)
scipy.sparse.save_npz('delta_s.npz', delta)

### Rank = 10

In [ ]:
r = rank = 10
Lambda_adaptive = 0.0001
Lambda_priorW = 0.01
epsilon   = 0.0000001
hy1=.001
A_L, B_L, C_L, W, delta = ADGNTD_stochastic_cp(Tens1, W, r = rank, iter_steps = [6000],
                      batchsize = [[20, 100, 20]], 
                      hy1= hy1, hy2=.0001, mu=0.1, factorization1=1000,
                      prin=5, A = None, B = None, C = None, W = None, delta = None )

In [8]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.1


# import cupy
print("TensorFlow version:", cupy.__version__)